In [1]:
!python -V

Python 3.9.5


In [124]:
import numpy as np
import pandas as pd

In [3]:
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [16]:
import os
os.getcwd()

'/home/valentin/lena/mlops-zoomcamp/01-intro'

In [128]:
df = pd.read_parquet('fhv_tripdata_2021-01.parquet')

df['duration'] = df.dropOff_datetime - df.pickup_datetime
df['duration'] = df['duration'].dt.total_seconds() / 60 

df = df[(df.duration >= 1) & (df.duration <= 60)]

df[['DOlocationID', 'PUlocationID']] = df[['DOlocationID', 'PUlocationID']].fillna(-1)
df[['DOlocationID', 'PUlocationID']] = df[['DOlocationID', 'PUlocationID']].astype(str)

In [129]:
df.shape

(1109826, 8)

In [130]:
df.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,-1.0,-1.0,None,B00009,17.000000
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,-1.0,-1.0,None,B00009,17.000000
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,-1.0,72.0,None,B00037,8.283333
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,-1.0,61.0,None,B00037,15.216667
5,B00037,2021-01-01 00:59:02,2021-01-01 01:08:05,-1.0,71.0,None,B00037,9.050000


In [131]:
df.isna().sum()/df.shape[0]

dispatching_base_num      0.000000
pickup_datetime           0.000000
dropOff_datetime          0.000000
PUlocationID              0.000000
DOlocationID              0.000000
SR_Flag                   1.000000
Affiliated_base_number    0.000697
duration                  0.000000
dtype: float64

In [132]:
df.duration.describe()

count    1.109826e+06
mean     1.624725e+01
std      1.155150e+01
min      1.000000e+00
25%      7.850000e+00
50%      1.323333e+01
75%      2.146667e+01
max      6.000000e+01
Name: duration, dtype: float64

In [133]:
df[["PUlocationID", "DOlocationID"]]

,PUlocationID,DOlocationID
0,-1.0,-1.0
1,-1.0,-1.0
3,-1.0,72.0
4,-1.0,61.0
5,-1.0,71.0
...,...,...
1154107,7.0,7.0
1154108,44.0,91.0
1154109,171.0,171.0
1154110,15.0,15.0


In [135]:
df.PUlocationID.describe()

count     1109826
unique        262
top          -1.0
freq       927008
Name: PUlocationID, dtype: object

In [136]:
df.DOlocationID.describe()

count     1109826
unique        263
top          -1.0
freq       147907
Name: DOlocationID, dtype: object

In [137]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1109826 entries, 0 to 1154111
Data columns (total 8 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   dispatching_base_num    1109826 non-null  object        
 1   pickup_datetime         1109826 non-null  datetime64[ns]
 2   dropOff_datetime        1109826 non-null  datetime64[ns]
 3   PUlocationID            1109826 non-null  object        
 4   DOlocationID            1109826 non-null  object        
 5   SR_Flag                 0 non-null        object        
 6   Affiliated_base_number  1109053 non-null  object        
 7   duration                1109826 non-null  float64       
dtypes: datetime64[ns](2), float64(1), object(5)
memory usage: 76.2+ MB


In [138]:
train_dicts = df[["PUlocationID", "DOlocationID"]].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = df[target].values

In [139]:
len(dv.vocabulary_)

525

In [140]:
len(dv.feature_names_)

525

In [141]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

10.528519403716007

In [146]:
df_feb = pd.read_parquet('fhv_tripdata_2021-02.parquet')

df_feb['duration'] = df_feb.dropOff_datetime - df_feb.pickup_datetime
df_feb['duration'] = df_feb['duration'].dt.total_seconds() / 60 

df_feb = df_feb[(df_feb.duration >= 1) & (df_feb.duration <= 60)]

df_feb[['DOlocationID', 'PUlocationID']] = df_feb[['DOlocationID', 'PUlocationID']].fillna(-1)
df_feb[['DOlocationID', 'PUlocationID']] = df_feb[['DOlocationID', 'PUlocationID']].astype(str)

X_feb_dicts = df_feb[["PUlocationID", "DOlocationID"]].to_dict(orient='records')
X_feb = dv.transform(X_feb_dicts)

In [147]:
y_feb = df_feb["duration"].values

In [148]:
y_pred_feb = lr.predict(X_feb)

mean_squared_error(y_feb, y_pred_feb, squared=False)

11.014287719752